In [19]:
import numpy as np
import os
import random
from random import shuffle
import scipy.misc
import scipy
import time
import skimage.io
from skimage.io import imread
import keras
import keras.callbacks
from keras import optimizers, metrics, regularizers
from keras import backend as K
from keras.models import Sequential, Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, SeparableConv2D, core
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Cropping2D, concatenate, Input
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.activations import softmax
import matplotlib
import sklearn.metrics
from sklearn.metrics import classification_report, confusion_matrix

class_weight_one = 1
class_weight_two = 1
batch_size = 2
epochs_2_train = 10
validPercent = 0.1
image_folder = '/data/Ro_ImageData/Low SN ratio ceramic images/' #Source of images
label_folder = '/data/Ro_ImageData/32xOTSU/' #Otsu images
prediction_folder = '/home/dspuser/MA499/Predictions/' #Folder for predictions

def get_sample(sample,image_folder,label_folder):
    inputImg = sample
    maskImg = sample.split('_C')[0] + '.tiff'
    img_int = np.array(imread(image_folder + inputImg), dtype = 'float32')/255
    lbl_int = np.array(imread(label_folder + maskImg), dtype = 'float32')/2**16
    lbl_int[:,:,1] = 1-lbl_int[:,:,0]
    lbl_int = lbl_int[:,:,0:2]
    return img_int,lbl_int

def get_sample_test(sample,test_folder):
    img_int = np.array(imread(test_folder + sample), dtype = 'float32')/255
    return img_int

def data_generator(file_name_list,batch_size): #randomly sampled instances from file name list with batch size
    while True:
        batch_filenames = np.random.choice(file_name_list,batch_size)
        batch_input = []
        batch_output = []
        for i_filename in batch_filenames:
            Ai_img, Ai_mask = get_sample(i_filename,image_folder,label_folder)
            batch_input += [Ai_img]
            batch_output += [Ai_mask]
            #print(i_filename)
        batch_img = np.expand_dims(np.array(batch_input,dtype='float32'),axis=-1)
        batch_mask = np.array(batch_output,dtype='float32')
        yield (batch_img,batch_mask)

def Weighted_Binary_CrossEntropy(y_true_n,y_pred_n):
    b_ce = K.binary_crossentropy(y_true_n,y_pred_n)
    y_true = K.cast(K.expand_dims(K.argmax(y_true_n,axis=-1),axis=-1),dtype='float32')
    y_pred = K.cast(K.expand_dims(K.argmax(y_pred_n,axis=-1),axis=-1),dtype='float32')
    #Pixel Disparity
    one_weight = class_weight_one
    zero_weight = class_weight_two
    weight_vector = y_true * one_weight + (1.-y_true)*zero_weight
    weighted_b_ce = weight_vector*b_ce
    return K.mean(weighted_b_ce)


sampleListAll = os.listdir(image_folder)

for file in sampleListAll:
    if os.path.splitext(file)[1] == '.tiff':
        sampleList.append(file)
        

    

indexes_valid = random.sample(range(0,len(sampleList)),int(validPercent*float(len(sampleList))))
indexes_train = [x for x in range(0,len(sampleList)) if x not in indexes_valid]
training_list = [x for ind,x in enumerate(sampleList) if ind in indexes_train]
valid_list = [x for ind,x in enumerate(sampleList) if ind in indexes_valid]
#(batch1,batch2) = data_generator(training_list,batch_size)
#print(batch1.shape)
#print(batch2.shape)

'''

test1,test2 = get_sample(training_list[0])
print(np.max(test2))
print(np.min(test2))
matplotlib.image.imsave('test1.png',test1)
matplotlib.image.imsave('test2.png',test2)

'''
num_train_calls = int(float(len(training_list)+1)/float(batch_size))
num_valid_calls = int(float(len(valid_list)+1)/float(batch_size))

#### Network is built ####
network = Sequential()
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),input_shape = (1024,1024,1),
padding = 'same'))
network.add(keras.layers.BatchNormalization())
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),padding='same'))
network.add(keras.layers.BatchNormalization())
network.add(Dropout(0.25))
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),padding='same'))
network.add(keras.layers.BatchNormalization())
network.add(Dropout(0.25))
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),padding='same'))
network.add(keras.layers.BatchNormalization())
network.add(Dropout(0.25))
network.add(Conv2D(2,(1,1),activation = 'softmax',kernel_regularizer = regularizers.l2(.01),padding='same'))
sgd = optimizers.SGD(lr = 0.0001, decay = 1e-8, momentum = 0.9, nesterov = False)
network.compile(loss = Weighted_Binary_CrossEntropy, optimizer = sgd, metrics = ['accuracy'])
network.summary()
#network.load_weights('weights.h5')
network_training = network.fit_generator(data_generator(training_list,batch_size),
steps_per_epoch = 50, epochs = epochs_2_train, verbose = 1,validation_data = 
data_generator(valid_list,batch_size),validation_steps = num_valid_calls)
#network.save_weights('weights.h5')
TP = 0 #True Positives
TN = 0 #True Negatives
FP = 0 #False Positives
FN = 0 #False Negatives

for filename in valid_list: #Use for new lists
    img,lbl = get_sample(filename,image_folder,label_folder)
    lbl = np.argmax(lbl,axis=-1)
    img = np.expand_dims(img,axis=0)
    img = np.expand_dims(img,axis=-1)
    img_pred = network.predict(img)
    pred = np.argmax(img_pred[0,:,:,:],axis=-1)
    TP += np.sum(pred*lbl,dtype='float32')
    TN += np.sum((1-pred)*(1-lbl),dtype='float32')
    FP += np.sum((1-pred)*lbl,dtype='float32')
    FN += np.sum(pred*(1-lbl),dtype='float32')

ACC = (TP + TN)/(TP + TN + FP + FN) #Accuracy
Recall = (TP)/(TP + FN)
Precision = (TP)/(TP + FP)
MCC = ((TP * TN) - (FP * FN))/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))

np.savetxt('ValidationMetrics2.txt',(ACC,Recall,Precision,MCC))
np.savetxt('ValidationLoss2.txt',network_training.history['val_loss'])
np.savetxt('ValidationAcc2.txt',network_training.history['val_accuracy'])

'''
plt.plot([1,2])
plt.ylabel(network_training.history['val_acc'])
plt.show()
'''

test_list = valid_list
'''
### Generate predictions from trained model ###
for filename in test_list:
    img,lbl = get_sample(filename,image_folder,label_folder)
    lbl = np.argmax(lbl,axis=-1)
    img = np.expand_dims(img,axis=0)
    img = np.expand_dims(img,axis=-1)
    img_pred = network.predict(img)
    pred = np.argmax(img_pred[0,:,:,:],axis=-1)
    matplotlib.image.imsave(prediction_folder+filename,pred)
'''

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 1024, 1024, 10)    100       
_________________________________________________________________
batch_normalization_37 (Batc (None, 1024, 1024, 10)    40        
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 1024, 1024, 10)    910       
_________________________________________________________________
batch_normalization_38 (Batc (None, 1024, 1024, 10)    40        
_________________________________________________________________
dropout_28 (Dropout)         (None, 1024, 1024, 10)    0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 1024, 1024, 10)    910       
_________________________________________________________________
batch_normalization_39 (Batc (None, 1024, 1024, 10)  

'\n### Generate predictions from trained model ###\nfor filename in test_list:\n    img,lbl = get_sample(filename,image_folder,label_folder)\n    lbl = np.argmax(lbl,axis=-1)\n    img = np.expand_dims(img,axis=0)\n    img = np.expand_dims(img,axis=-1)\n    img_pred = network.predict(img)\n    pred = np.argmax(img_pred[0,:,:,:],axis=-1)\n    matplotlib.image.imsave(prediction_folder+filename,pred)\n'

In [17]:
prediction_folder = '/home/dspuser/MA499/Predictions/' #Folder for predictions
test_list = valid_list
for filename in test_list:
    img,lbl = get_sample(filename,image_folder,label_folder)
    lbl = np.argmax(lbl,axis=-1)
    img = np.expand_dims(img,axis=0)
    img = np.expand_dims(img,axis=-1)
    img_pred = network.predict(img)
    pred = np.argmax(img_pred[0,:,:,:],axis=-1)
    matplotlib.image.imsave(prediction_folder+filename,pred)

In [18]:
img_pred

array([[[[4.08953913e-02, 9.59104598e-01],
         [2.86899805e-01, 7.13100255e-01],
         [2.55960273e-04, 9.99743998e-01],
         ...,
         [5.55715133e-07, 9.99999404e-01],
         [2.59438650e-07, 9.99999762e-01],
         [1.01332866e-01, 8.98667097e-01]],

        [[1.43952379e-02, 9.85604703e-01],
         [1.61494652e-04, 9.99838471e-01],
         [3.84293380e-04, 9.99615669e-01],
         ...,
         [4.71111089e-02, 9.52888906e-01],
         [1.61086628e-08, 1.00000000e+00],
         [4.65085894e-01, 5.34914076e-01]],

        [[9.20047518e-04, 9.99079943e-01],
         [9.09818470e-01, 9.01815668e-02],
         [1.36495659e-08, 1.00000000e+00],
         ...,
         [2.37011317e-08, 1.00000000e+00],
         [8.35604386e-09, 1.00000000e+00],
         [4.16931385e-07, 9.99999642e-01]],

        ...,

        [[8.59623123e-03, 9.91403759e-01],
         [7.50332233e-03, 9.92496729e-01],
         [4.41316705e-11, 1.00000000e+00],
         ...,
         [8.85718942e